<a href="https://colab.research.google.com/github/BrianSantos66/challenge-datascience-v2/blob/main/Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação das dependências

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
import findspark
import zipfile
from google.colab import drive

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [4]:
findspark.init()

## Criação da seção e montagem do drive

In [91]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType, IntegerType, LongType
from pyspark.sql import functions as f

In [6]:
spark = SparkSession.builder\
     .master('local[*]')\
    .appName("Iniciando com Spark")\
    .config("spark.ui.port", '4050')\
    .getOrCreate()

In [7]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
get_ipython().system_raw('./ngrok config add-authtoken 2IGFrcHdmscOt9I9zJ1Zsxvmddj_4ECf4Sh3SML7wuPi5A5v1')
get_ipython().system_raw('./ngrok http 4050 &')

In [9]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://d29c-35-199-24-62.ngrok-free.app","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://d29c-35-199-24-62.ngrok-free.app","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [10]:
drive.mount('/content/drive')

Mounted at /content/drive


## Carregando os dados


In [11]:
zipfile.ZipFile("/content/drive/MyDrive/desafio_datascience/semana-1.zip", "r").extractall("/content/drive/MyDrive/desafio_datascience/semana-1/")

In [77]:
path = "/content/drive/MyDrive/desafio_datascience/semana-1/dataset_bruto.json"
dados = spark.read.json(path)

# Exploração dos dados

In [13]:
print(f"Quantidade de linhas do datafame: {dados.count()}")

Quantidade de linhas do datafame: 89083


In [14]:
dados.limit(5).show(truncate= False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------+
|anuncio                                                               

In [15]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

Talvez seja necessário converter:

- anuncio.id para inteiro (string atualmente)
- anuncio.valores.valor para inteiro (string atualmente)
- anuncio.valores.iptu para inteiro (string atualmente)
- imagens.id para inteiro (string atualmente)
- usuario.id para inteiro (string atualmente)

## Criando um dataframe a parte com as informações de anuncio

In [78]:
df_anuncio = dados\
              .select("anuncio")
df_anuncio.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [79]:
df_anuncio = df_anuncio\
                  .withColumn("andar", df_anuncio.anuncio.andar)\
                  .withColumn("area_total", df_anuncio.anuncio.area_total)\
                  .withColumn("area_util", df_anuncio.anuncio.area_util)\
                  .withColumn("banheiros", df_anuncio.anuncio.area_util)\
                  .withColumn("caracteristicas", df_anuncio.anuncio.caracteristicas)\
                  .withColumn("bairro", df_anuncio.anuncio.endereco.bairro)\
                  .withColumn("cep", df_anuncio.anuncio.endereco.cep)\
                  .withColumn("cidade", df_anuncio.anuncio.endereco.cidade)\
                  .withColumn("estado", df_anuncio.anuncio.endereco.estado)\
                  .withColumn("latitude", df_anuncio.anuncio.endereco.latitude)\
                  .withColumn("longitude", df_anuncio.anuncio.endereco.longitude)\
                  .withColumn("pais", df_anuncio.anuncio.endereco.pais)\
                  .withColumn("rua", df_anuncio.anuncio.endereco.rua)\
                  .withColumn("zona", df_anuncio.anuncio.endereco.zona)\
                  .withColumn("id", df_anuncio.anuncio.id.cast("long"))\
                  .withColumn("quartos", df_anuncio.anuncio.quartos)\
                  .withColumn("suites", df_anuncio.anuncio.suites)\
                  .withColumn("tipo_anuncio", df_anuncio.anuncio.tipo_anuncio)\
                  .withColumn("tipo_unidade", df_anuncio.anuncio.tipo_unidade)\
                  .withColumn("tipo_uso", df_anuncio.anuncio.tipo_uso)\
                  .withColumn("vaga", df_anuncio.anuncio.vaga)\
                  .withColumn("valor_condominio", df_anuncio.anuncio.valores.condominio)\
                  .withColumn("valor_iptu", df_anuncio.anuncio.valores.iptu)\
                  .withColumn("valor_tipo", df_anuncio.anuncio.valores.tipo)\
                  .withColumn("valor", df_anuncio.anuncio.valores.valor)

df_anuncio = df_anuncio.drop(df_anuncio.anuncio)
df_anuncio.show()


+-----+----------+---------+---------+--------------------+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|        bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|        zona|  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor_condominio|valor_iptu|valor_tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------+--------+--------------+--------------+----------+----------+----+--------------------+------------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|    0|        []|     [16]|     [16]|                  []|        Centro|20061003|Rio de Janeiro|Rio de Janeiro|-22.906082| -43.18671|  BR|    R

## Frequência e filtro das coluna tipo_uso, tipo_unidade e tipo_anuncio

In [80]:
# Frequência da coluna tipo_uso
df_anuncio\
  .select("tipo_uso")\
  .groupBy("tipo_uso")\
  .agg(
      f.count("tipo_uso").alias("frequencia")
      )\
  .show()

# Fazer uma nova coluna com o percentual de cada tipo

+-----------+----------+
|   tipo_uso|frequencia|
+-----------+----------+
|  Comercial|      4542|
|Residencial|     84541|
+-----------+----------+



In [81]:
comercial = 4542
residencial = 84541
total = comercial + residencial

print(f"Comercial: {(comercial / total)*100:.2f}%, Residencial: {(residencial / total)*100:.2f}%")

Comercial: 5.10%, Residencial: 94.90%


- É possivel observar que a base possui mais imóveis do tipo Residencial (94.90%), enquanto o tipo Comercial equivale a 5.10% dos dados

In [82]:
# Frequência da coluna tipo_unidade
df_anuncio\
  .select("tipo_unidade")\
  .groupBy("tipo_unidade")\
  .agg(
      f.count("tipo_unidade").alias("Frequência")
      )\
  .show()

+------------+----------+
|tipo_unidade|Frequência|
+------------+----------+
|      Outros|     11963|
| Apartamento|     66801|
|        Casa|     10319|
+------------+----------+



In [83]:
outros = 11963
apartamento = 66801
casa = 10319
total = outros + apartamento + casa

print(f"Outros: {(outros / total)*100:.2f}%, Apartamento: {(apartamento / total)*100:.2f}%, Casa: {(casa / total)*100:.2f}%")

Outros: 13.43%, Apartamento: 74.99%, Casa: 11.58%


- O tipo de unidade que mais aparece é Apartamento (74.99%), a base apresenta mais dados com o tipo "Outros" (13.43%) do que com o tipo "Casa" (11.58%)

In [84]:
# Frequência da coluna tipo_anuncio
df_anuncio\
  .select("tipo_anuncio")\
  .groupBy("tipo_anuncio")\
  .agg(
      f.count("tipo_anuncio").alias("Frequência")
      )\
  .show()

+------------+----------+
|tipo_anuncio|Frequência|
+------------+----------+
|       Usado|     88827|
|  Lançamento|       256|
+------------+----------+



In [85]:
usado = 88827
lançamento = 256
total = usado + lançamento

print(f"Usado: {(usado / total)*100:.2f}%, Lançamento: {(lançamento / total)*100:.2f}%")

Usado: 99.71%, Lançamento: 0.29%


- 99.71% dos tipos dos anuncios são de imóveis usados, enquanto imóveis em lançamento correspondem a 0.29%

In [86]:
# Filtrando a base procurando pelos tipos residencial, apartamento e usado
df_filtro = df_anuncio\
  .where(f.upper(df_anuncio.tipo_uso).like("RESIDENCIAL"))\
  .where(f.upper(df_anuncio.tipo_unidade).like("APARTAMENTO"))\
  .where(f.upper(df_anuncio.tipo_anuncio).like("USADO"))

df_filtro.show()

+-----+----------+---------+---------+--------------------+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor_condominio|valor_iptu|valor_tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|    3|      [43]|     [43]|     [43]|[Academia, Churra...|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43

## Trabalhando com o DataFrame filtrado

- Optei por utilizar apenas os dados que se encontravam na primeira posição da lista em cada coluna

In [ ]:
df_filtro = df_filtro\
  .withColumn("quartos", df_filtro.quartos[0].cast(IntegerType()))\
  .withColumn("suites", df_filtro.suites[0].cast(IntegerType()))\
  .withColumn("banheiros", df_filtro.banheiros[0].cast(IntegerType()))\
  .withColumn("vaga", df_filtro.vaga[0].cast(IntegerType()))\
  .withColumn("area_total", df_filtro.area_total[0].cast(IntegerType()))\
  .withColumn("area_util", df_filtro.area_util[0].cast(IntegerType()))\

df_filtro.printSchema()

In [101]:
df_filtro.show()

+-----+----------+---------+---------+--------------------+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|andar|area_total|area_util|banheiros|     caracteristicas|              bairro|     cep|        cidade|        estado|  latitude| longitude|pais|                 rua|      zona|  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|valor_condominio|valor_iptu|valor_tipo|  valor|
+-----+----------+---------+---------+--------------------+--------------------+--------+--------------+--------------+----------+----------+----+--------------------+----------+----+-------+------+------------+------------+-----------+----+----------------+----------+----------+-------+
|    3|        43|       43|       43|[Academia, Churra...|           Paciência|23585430|Rio de Janeiro|Rio de Janeiro|-22.919851|-43

### Informações sobre endereço (dados filtrados)

In [103]:
df_localizacao = df_filtro\
                .select("bairro", "zona")
df_localizacao.show()   

+--------------------+----------+
|              bairro|      zona|
+--------------------+----------+
|           Paciência|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|           Paciência|Zona Oeste|
|           Guaratiba|Zona Oeste|
|  Pedra de Guaratiba|Zona Oeste|
|              Cosmos|Zona Oeste|
|        Campo Grande|Zona Oeste|
|          Santa Cruz|Zona Oeste|
|           Guaratiba|Zona Oeste|
|Recreio dos Bande...|Zona Oeste|
|              Cosmos|Zona Oeste|
|            Realengo|Zona Oeste|
|              Cosmos|Zona Oeste|
|     Todos os Santos|Zona Norte|
+--------------------+----------+
only showing top 20 rows



### Informações sobre valores (dados filtrados)

In [105]:
df_valores = df_filtro\
              .select("valor_condominio", "valor_iptu", "valor_tipo", "valor")

df_valores = df_valores\
              .withColumn("valor_condominio", df_valores.valor_condominio[0].cast(IntegerType()))\
              .withColumn("valor_iptu", df_valores.valor_iptu[0].cast(IntegerType()))\
              .withColumn("valor_tipo", df_valores.valor_tipo[0])\
              .withColumn("valor", df_valores.valor[0].cast(IntegerType()))
              
df_valores.show()

+----------------+----------+----------+-----+
|valor_condominio|valor_iptu|valor_tipo|valor|
+----------------+----------+----------+-----+
|             245|      null|     Venda|15000|
|               0|         0|     Venda|15000|
|               0|         0|     Venda|20000|
|             285|      null|     Venda|20000|
|             245|      null|     Venda|15000|
|             285|      null|     Venda|20000|
|             250|      null|     Venda|15000|
|             245|      null|     Venda|15000|
|             245|      null|     Venda|15000|
|             240|      null|     Venda|17999|
|               0|         0|     Venda|15000|
|             240|         0|     Venda|19999|
|             245|      null|     Venda|19999|
|             290|      null|     Venda|25000|
|             285|      null|     Venda|29999|
|               0|         0|     Venda|30000|
|            null|      null|     Venda|30000|
|             280|         0|     Venda|25000|
|            

In [110]:
# Filtrando por "Venda" na coluna valor_tipo
df_valores\
  .where("valor_tipo = 'Venda'").show()

+----------------+----------+----------+-----+
|valor_condominio|valor_iptu|valor_tipo|valor|
+----------------+----------+----------+-----+
|             245|      null|     Venda|15000|
|               0|         0|     Venda|15000|
|               0|         0|     Venda|20000|
|             285|      null|     Venda|20000|
|             245|      null|     Venda|15000|
|             285|      null|     Venda|20000|
|             250|      null|     Venda|15000|
|             245|      null|     Venda|15000|
|             245|      null|     Venda|15000|
|             240|      null|     Venda|17999|
|               0|         0|     Venda|15000|
|             240|         0|     Venda|19999|
|             245|      null|     Venda|19999|
|             290|      null|     Venda|25000|
|             285|      null|     Venda|29999|
|               0|         0|     Venda|30000|
|            null|      null|     Venda|30000|
|             280|         0|     Venda|25000|
|            

## Salvando/Criando Arquivos

In [111]:
df_valores.write.parquet(
    path="/content/drive/MyDrive/desafio_datascience/semana-1/parquet",
    mode= "overwrite"
)

In [112]:
df_valores.write.csv(
    path="/content/drive/MyDrive/desafio_datascience/semana-1/csv",
    mode= "overwrite",
    sep= ";",
    header= True
)

### Comparação de desempenho de leitura

In [113]:
%%time
path = "/content/drive/MyDrive/desafio_datascience/semana-1/csv"
dados = spark.read.csv(path)

CPU times: user 6.71 ms, sys: 1.95 ms, total: 8.66 ms
Wall time: 357 ms


In [114]:
%%time
path = "/content/drive/MyDrive/desafio_datascience/semana-1/parquet"
dados = spark.read.parquet(path)

CPU times: user 4.42 ms, sys: 156 µs, total: 4.58 ms
Wall time: 200 ms


- Comprova que a leitura de arquivos parquet é mais eficiente